In [11]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split

In [13]:
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [15]:
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split(Auto ,
test_size=196,
random_state=0)

In [17]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [19]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [25]:
def evalMSE(terms ,
    response ,
    train ,
    test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [27]:
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
    'mpg',
    Auto_train ,
    Auto_valid)
    MSE

In [31]:
Auto_train , Auto_valid = train_test_split(Auto ,
test_size=196,
random_state=3)
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
    'mpg',
    Auto_train ,
    Auto_valid)
    MSE

In [33]:
hp_model = sklearn_sm(sm.OLS ,
MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model ,
X,
Y,
cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

In [35]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
    X,
    Y,
    cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
    cv_error

In [37]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [41]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
shuffle=True ,
random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
    X,
    Y,
    cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
    cv_error

In [43]:
validation = ShuffleSplit(n_splits=1,
test_size=196,
random_state=0)
results = cross_validate(hp_model ,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation);
results['test_score']

array([23.61661707])

In [45]:
validation = ShuffleSplit(n_splits=10,
test_size=196,
random_state=0)
results = cross_validate(hp_model ,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.421845094109187)

In [51]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
    (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))
    alpha_func(Portfolio , range(100))

In [53]:
rng = np.random.default_rng(0)
alpha_func(Portfolio ,
rng.choice(100,
100,
replace=True))

0.6074452469619002

In [65]:
import numpy as np

def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0.0, 0.0

    # If n is not provided, use sample size of D
    n = n or D.shape[0]

    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value ** 2

    return np.sqrt(second_ / B - (first_ / B) ** 2)


In [67]:
alpha_SE = boot_SE(alpha_func ,
Portfolio ,
B=1000,
seed=0)
alpha_SE

0.09118176521277577

In [79]:

def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

# hp_func will take (D, idx)
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

rng = np.random.default_rng(0)

# IMPORTANT: sample from Auto.index (not range(392))
res = np.array([hp_func(Auto, rng.choice(Auto.index, size=len(Auto), replace=True))
                for _ in range(10)])

res


array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

In [81]:
hp_se = boot_SE(hp_func ,
Auto ,
B=1000,
seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

In [83]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS ,
quad_model ,
'mpg')
boot_SE(quad_func , Auto , B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [85]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64